# **요약**
* **간단한 답변**
    1. LLM 설정
    2. LLM.invoke(question)
    
* **LLM Chain을 사용한 답변**
    1. LLM 설정
    2. 문자열로 template 작성 ({}를 사용해 input_variables 지정)
    3. template으로 PromptTemplate 생성
    4. LLMChain(prompt=prompt, llm=llm) 생성
    5. 답변 받아오기
        a. invoke(): 단일 질문
        b. apply(): 복수 질문 (리스트 형태)
        c. generate(): 복수 질문 + 메타 데이터

* **streaming** 
    1. 답변을 받는 방법은 위와 동일하나 LLM 설정 단계가 다름
        a. streaming=True
        b. callbacks=[StreamingStdOutCallbackHandler()]

### 버전 확인

In [3]:
import openai

openai.__version__

'1.13.3'

### ChatOpenAI

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    temperature = 0.1,
    max_tokens=2048,
    model_name='gpt-3.5-turbo',
)

question = "대한민국의 수도는? (짦게 답변)"

print(f"[답변]: {llm.invoke(question)}")

[답변]: content='서울입니다.' response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 27, 'total_tokens': 32}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None}


### 프롬프트 템플릿의 활용
**PromptTemplate**
* 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
* 사용법
    * template: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 {}는 변수를 나타냅니다.
    * input_variables: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

**input_variables**
* input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.
* 사용법: 리스트 형식으로 변수 이름을 정의합니다.

In [1]:
from langchain.prompts import PromptTemplate

template = "{country}의 수도는 뭐야?"

prompt = PromptTemplate.from_template(template=template)
prompt

PromptTemplate(input_variables=['country'], template='{country}의 수도는 뭐야?')

### LLMChain 객체
**LLMChain**
* LLMChain은 특정 <U>PromptTemplate</U>와 연결된 체인 객체를 생성합니다
* 사용법
    * prompt: 앞서 정의한 PromptTemplate 객체를 사용합니다.
    * llm: 언어 모델을 나타내며, 이 예시에서는 이미 어딘가에서 정의된 것으로 보입니다.

In [9]:
from langchain.chains import LLMChain

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [10]:
llm_chain.invoke({"country": "대한민국"})

{'country': '대한민국', 'text': '대한민국의 수도는 서울이야.'}

In [11]:
llm_chain.invoke({"country": "캐나다"})

{'country': '캐나다', 'text': '캐나다의 수도는 오타와(Ottawa)입니다.'}

**apply()**
* 여러개의 입력에 대한 처리를 한 번에 수행할 수 있다.

In [12]:
input_list = [{"country": "호주"}, {"country": "중국"}, {"country": "네덜란드"}]

response = llm_chain.apply(input_list)

In [16]:
for res in response:
    print(res['text'].strip())

호주의 수도는 캔버라입니다.
중국의 수도는 베이징(北京)입니다.
네덜란드의 수도는 암스테르담입니다.


**generate()**
* LLMResult를 반환하는 점을 제외하고는 apply와 유사하다.
* 토큰사용량과 종료 이유와 같은 유용한 생성 정보를 포함한다.

In [17]:
generated_result = llm_chain.generate(input_list)
print(generated_result)

generations=[[ChatGeneration(text='호주의 수도는 캔버라입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='호주의 수도는 캔버라입니다.', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 17, 'total_tokens': 30}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None}))], [ChatGeneration(text='중국의 수도는 베이징(北京)입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='중국의 수도는 베이징(北京)입니다.', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 19, 'total_tokens': 37}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None}))], [ChatGeneration(text='네덜란드의 수도는 암스테르담입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='네덜란드의 수도는 암스테르담입니다.', response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 22, 'total_tokens'

In [18]:
generated_result.llm_output

{'token_usage': {'completion_tokens': 53,
  'prompt_tokens': 58,
  'total_tokens': 111},
 'model_name': 'gpt-3.5-turbo',
 'system_fingerprint': 'fp_3bc1b5746c'}

### input_variables 여러 개 사용 예제

In [21]:
template = "{area1} 와 {area2} 의 시차는 몇 시간이야?"

prompt = PromptTemplate.from_template(template=template)
prompt

PromptTemplate(input_variables=['area1', 'area2'], template='{area1} 와 {area2} 의 시차는 몇 시간이야?')

In [24]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
llm_chain

LLMChain(prompt=PromptTemplate(input_variables=['area1', 'area2'], template='{area1} 와 {area2} 의 시차는 몇 시간이야?'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000232DD01C520>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000232DD02E130>, temperature=0.1, openai_api_key=SecretStr('**********'), openai_proxy='', max_tokens=2048))

In [25]:
print(llm_chain.invoke({"area1": "서울", "area2": "파리"}))

{'area1': '서울', 'area2': '파리', 'text': '서울과 파리의 시차는 8시간입니다. 서울은 GMT+9 시간대에 속하고, 파리는 GMT+1 시간대에 속하기 때문에 시차가 8시간이 발생합니다.'}


In [26]:
input_list = [
    {"area1": "파리", "area2": "뉴욕"},
    {"area1": "서울", "area2": "하와이"},
    {"area1": "켄버라", "area2": "베이징"},
]

result = llm_chain.apply(input_list)
for res in result:
    print(res["text"].strip())

파리와 뉴욕의 시차는 6시간입니다. 파리는 그리니치 평균시(GMT+1)를 따르고, 뉴욕은 동부 표준시(EST 또는 GMT-5)를 따르기 때문에 시차가 발생합니다.
서울과 하와이의 시차는 서울이 하와이보다 19시간 빠릅니다. 서울은 GMT+9 시간대에 위치하고 있으며, 하와이는 GMT-10 시간대에 위치하고 있기 때문입니다.
켄버라와 베이징의 시차는 2시간입니다. 켄버라는 UTC+10 시간대에 있고, 베이징은 UTC+8 시간대에 있기 때문입니다.


### stream: 실시간 출력
* 스트리밍 옵션은 질의에 대한 답변을 실시간으로 받을 때 유용하다.
* 다음과 같이 **streaming=True**로 설정하고 스트리밍으로 답변을 받기 위한
**StreamingStdOutCallbackHandler()** 을 콜백으로 지정한다.

In [28]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = ChatOpenAI(
    temperature=0,
    max_tokens=2048,
    model_name="gpt-3.5-turbo",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

In [29]:
question = "대한민국에 대해서 간략하게 20자 내외로 알려줘"

response = llm.invoke(question)

한반도 동쪽에 위치한 나라로, 수도는 서울이며 한국어를 사용하고 있습니다. 현대화된 경제와 문화를 갖추고 있습니다.